## Carregando os modelos treinados a partir da pasta artifacts

Nesta etapa vamos configurar o notebook para carregar automaticamente os modelos treinados que foram salvos na pasta artifacts. A ideia é percorrer as subpastas HIST, MLP e TREEDECISION, localizar os arquivos .pkl correspondentes ao modelo final e carregá-los em memória. Com isso, conseguimos reutilizar os modelos já treinados sem precisar refazer o processo de treinamento dentro deste notebook.


In [1]:
from pathlib import Path
import joblib
import pandas as pd

DATA_PATH = "../data/gold/salaries_full.parquet"
ARTIFACTS_DIR = Path("../artifacts")

df_full = pd.read_parquet(DATA_PATH)

features_finais = [
    "experience_level_encoded",
    "company_size_encoded",
    "remote_ratio",
    "is_US",
    "job_type_Full-time",
    "job_type_Contract",
    "job_type_Part-time",
]

features_presentes = [c for c in features_finais if c in df_full.columns]
X_full = df_full[features_presentes].astype(float)

print("Features usadas no carregamento:", features_presentes)
print("Formato de X_full:", X_full.shape)


Features usadas no carregamento: ['experience_level_encoded', 'company_size_encoded', 'remote_ratio', 'is_US', 'job_type_Full-time', 'job_type_Contract', 'job_type_Part-time']
Formato de X_full: (22789, 7)


## Localizando e carregando os arquivos .pkl de cada modelo

Aqui percorremos as subpastas dentro de artifacts e buscamos os arquivos .pkl finais de cada tipo de modelo. Para simplificar, usamos o nome da pasta como identificador do modelo (mlp, hist, treedecision) e carregamos o primeiro .pkl encontrado em cada uma delas. No fim, expomos variáveis separadas para facilitar o uso durante a inferência.


In [3]:
models = {}

for subdir in ARTIFACTS_DIR.iterdir():
    if subdir.is_dir():
        pkl_files = list(subdir.glob("*.pkl"))
        if not pkl_files:
            continue
        models[subdir.name.lower()] = joblib.load(pkl_files[0])

model_mlp = models.get("mlp")
model_hist = models.get("hist")
model_tree = models.get("tree_decision")

print("Modelos carregados:")
print("MLP:", type(model_mlp))
print("Hist:", type(model_hist))
print("tree_decision:", type(model_tree))


Modelos carregados:
MLP: <class 'sklearn.pipeline.Pipeline'>
Hist: <class 'sklearn.ensemble._hist_gradient_boosting.gradient_boosting.HistGradientBoostingRegressor'>
tree_decision: <class 'sklearn.tree._classes.DecisionTreeRegressor'>


## Criando um exemplo de entrada para testar a inferência

Para comparar a predição dos três modelos, vamos montar um pequeno conjunto de exemplos artificiais com as mesmas colunas usadas no treinamento. Esses exemplos representam cenários típicos de combinações de experiência, tamanho de empresa, remoto e tipo de vínculo. Em seguida, iremos passar esse mesmo conjunto de entrada para todos os modelos e observar os salários previstos.


In [10]:
X_example = pd.DataFrame(
    [
        {
            "experience_level_encoded": 1,
            "company_size_encoded": 1,
            "remote_ratio": 0,
            "is_US": 0,
            "job_type_Full-time": True,
            "job_type_Contract": False,
            "job_type_Part-time": False,
        },
        {
            "experience_level_encoded": 2,
            "company_size_encoded": 2,
            "remote_ratio": 50,
            "is_US": 1,
            "job_type_Full-time": True,
            "job_type_Contract": False,
            "job_type_Part-time": False,
        },
        {
            "experience_level_encoded": 3,
            "company_size_encoded": 3,
            "remote_ratio": 100,
            "is_US": 1,
            "job_type_Full-time": True,
            "job_type_Contract": False,
            "job_type_Part-time": False,
        },
    ],
    columns=features_presentes,
).astype(float)

print(X_example)


   experience_level_encoded  company_size_encoded  remote_ratio  is_US  \
0                       1.0                   1.0           0.0    0.0   
1                       2.0                   2.0          50.0    1.0   
2                       3.0                   3.0         100.0    1.0   

   job_type_Full-time  job_type_Contract  job_type_Part-time  
0                 1.0                0.0                 0.0  
1                 1.0                0.0                 0.0  
2                 1.0                0.0                 0.0  


## Realizando a inferência com os três modelos para comparação

Agora utilizamos o mesmo conjunto de exemplos para obter as previsões de cada modelo carregado. A saída é organizada em um DataFrame que mostra, lado a lado, as previsões do MLP, do modelo de HistGradientBoosting e do modelo de árvore de decisão. Isso permite comparar de forma direta como cada abordagem responde ao mesmo perfil de profissional e vaga.


In [16]:
preds = {}

if model_mlp is not None:
    preds["MLP"] = model_mlp.predict(X_example)

if model_hist is not None:
    preds["HIST"] = model_hist.predict(X_example)

if model_tree is not None:
    preds["TREE_DECISION"] = model_tree.predict(X_example)

df_preds = pd.DataFrame(preds)
df_result = pd.concat([X_example.reset_index(drop=True), df_preds], axis=1)

df_result


,experience_level_encoded,company_size_encoded,remote_ratio,is_US,job_type_Full-time,job_type_Contract,job_type_Part-time,MLP,HIST,TREE_DECISION
0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,65696.127157,54913.922261,49178.857143
1,2.0,2.0,50.0,1.0,1.0,0.0,0.0,83071.280772,102885.354281,102617.587613
2,3.0,3.0,100.0,1.0,1.0,0.0,0.0,133900.712671,137479.092308,138066.735849
